# Agents

The agents are intelligent entities and components interacting with the language model and other tools to fulfill a task. They have the capability to select the appropriate tools from a given list or formulate a plan and incorporate suitable tools at each step to get the job done. 

* [1. What are Agents](#what)
    * [1.1. "Content Generator"- Writing a short science fiction story based on a given prompt](#generator)
    * [1.2. "Reasoning Engine" - Answering Questions using Google Search and Math tool](#reasoning)
* [2. Autonomous Agents (AutoGPT and BabyAGI)](#autonomous)
* [3. Using AutoGPT with LangChain](#AutoGPT)
* [4. Agent Simulation Projects: CAMEL and Generative Agents](#simulation)
* [5. "Plan and Execute" framework](#plan_and_execute)
* [6. Current Insights and Trends of Agents](#curr)

In [1]:
import os
from keys import OPENAI_API_KEY, GOOGLE_API_KEY, GOOGLE_CSE_ID

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["GOOGLE_CSE_ID"] = GOOGLE_CSE_ID

In [2]:
#!pip install langchain==0.0.208 deeplake openai tiktoken

<hr>
<a class="anchor" id="what">
    
## 1. What are Agents
    
</a>

Agents - intelligent systems that utilize LLMs to determine actions and facilitate complex tasks. In the context of LLMs, agents are used to decide the course of action and the sequence of these actions. These actions can either be the utilization of a tool, observing its output, or offering a response to the user. 

- **Tool**: A function that performs a specific task. It can be a Google Search, a Database lookup, a Python REPL, or other chains. A tool's interface is typically a function that takes a string as an input and returns a string as an output.
- **Language Learning Model**: The language model that powers the agent.
- **Agent**: The agent to use, identified by a string that references a supported agent class. It’s what orchestrates the LLM and the tools. 

Agents in LangChain are of two categories:
- **"Action Agents"**: typically used for straightforward tasks - they determine and execute a single action. 
- **"Plan-and-Execute Agents"**: first devise a plan comprising multiple actions and then execute each action sequentially. They are more suited for complex or long-running tasks as they help maintain focus on long-term objectives. (Be caucious - they might also lead to more calls and higher latency).

Workflow of Action Agents:
1. The agent receives user input.
2. It decides which tool to use (if any) and determines its input.
3. The chosen tool is called with the provided input, and an observation (the output of the tool) is recorded.
4. The history of the tool, tool input, and observation are relayed back to the agent, which then decides the next step.
5. This process is repeated until the agent no longer needs to use a tool, at which point it directly responds to the user.

Workflow when the agent functions as a "reasoning engine":

1. Query Processing: The agent receives a query and identifies distinct tasks within this query;
2. Tool Utilization: The agent uses tools  - for example, "google-search" tool to answer the part of the query where necessary. The agent isn't creating this information; it's pulling the data from an external source.
3. Information Processing: Again, the agent isn't creating new information. Instead, it's processing the data it has gathered. For example, it can use the "llm-math" tool to perform a sum reliably.
4. Synthesis and Response: After gathering and processing the information, the agent synthesizes it into a coherent response that answers the original query.

<div class="alert alert-block alert-success">
In the context of LangChain, the term "agents" pertains to the concept of employing a language model as a <b>reasoning mechanism</b> and linking it with the key element - a tool. Tools are instrumental in connecting the language model with other sources of data or computation, including search engines, APIs, and other data repositories. Tools can also execute actions (like running code or modifying files) and observe the results, subsequently informing the language model's decision-making process.
</div>

<hr>
<a class="anchor" id="generator">
    
### 1.1. "Content Generator"- Writing a short science fiction story based on a given prompt
</a>

In [3]:
# Importing necessary modules
from langchain.agents import initialize_agent, AgentType
from langchain.llms import OpenAI
from langchain.agents import Tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["query"],
    template="You're a renowned science fiction writer. {query}"
)

# Initialize the language model
llm = OpenAI(model="text-davinci-003", temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)

tools = [
    Tool(
        name='Science Fiction Writer',
        func=llm_chain.run,
        description='Use this tool for generating science fiction stories. Input should be a command about generating specific types of stories.'
    )
]

# Initializing an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Testing the agent with the new prompt
response = agent.run("Compose an epic science fiction saga about interstellar explorers")
print(response)



> Entering new AgentExecutor chain...
 I should use a tool to help me generate a story
Action: Science Fiction Writer
Action Input: Generate a story about interstellar explorers
Observation: .

The year was 2120 and the human race had finally achieved interstellar travel. After decades of research and development, a team of brave explorers had been assembled to embark on a mission to explore the unknown reaches of the universe.

The team was led by Captain John Smith, a veteran of many space missions. He was joined by a diverse group of scientists, engineers, and adventurers, all eager to explore the unknown.

The team's first destination was a distant star system known as Alpha Centauri. After months of travel, they arrived at the system and began to explore the planets and moons in the vicinity.

The team discovered a number of strange and fascinating phenomena, from strange alien life forms to mysterious energy fields. They also encountered a number of hostile forces, from hostile

**NOTE**: LLM's understanding comes from its training data: it was trained on a diverse range of texts, so in this case its responses are generated based on patterns learned during training, not from a specific knowledge database. (Using LLM as a content creator can sometimes result in "hallucinations" or inaccuracies).

<hr>
<a class="anchor" id="reasoning">
    
### 1.2. "Reasoning Engine" - Answering Questions using Google Search and Math tool
</a>

In [4]:
# Importing necessary modules
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

# Loading the language model to control the agent
llm = OpenAI(model="text-davinci-003", temperature=0)

# Loading some tools to use. The llm-math tool uses an LLM, so we pass that in.
tools = load_tools(["google-search", "llm-math"], llm=llm)

# Initializing an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Testing the agent
query = "What's the result of 1000 plus the number of goals scored in the soccer world cup in 2018?"
response = agent.run(query)
print(response)



> Entering new AgentExecutor chain...
 I need to find out the number of goals scored in the 2018 soccer world cup
Action: google_search
Action Input: number of goals scored in the 2018 soccer world cup
Observation: This was three more goals than the previous tournament in 2018. Goals galore. The highest ever goals to game ratio was set during the 1954 World Cup in ... Jan 31, 2020 ... A total of 169 goals were scored at the group and knockout stages of the FIFA World Cup held in Russia from the 14th of June to the 15th of July ... 3 days ago ... This number can be attributed to some unusually high-scoring games, which included Austria 7 Switzerland 5 and Hungary 9 South Korea 0. The ... Number of goals scored in the matches played between the teams in question;; Fair play points in all group matches (only one deduction could be applied to a ... France were crowned champions for the second time in history and for the first since they were hosts in 1998 after defeating Croatia 4-2 in w

<hr>
<a class="anchor" id="autonomous">
    
## 2. Autonomous Agents (AutoGPT and BabyAGI)
    
</a>

Autonomous agents are AI systems designed to carry out tasks without needing constant human guidance. These agents are making waves due to their ability to work independently to achieve a specific objective. 

**AutoGPT** is an open-source initiative which employs GPT-4 to sift through the internet in a structured manner, formulate subtasks, and initiate new agents. AutoGPT can understand the overall goal, break it down into subtasks, execute those tasks, and dynamically adjust its actions based on the ongoing context.

The initial conception of AutoGPT was as a general autonomous agent capable of doing anything. However, the wide breadth of application seemed to dilute its effectiveness and, as a result, a shift has been observed in the AutoGPT space, with developers starting to build specialized agents. These agents are designed to perform specific tasks effectively and efficiently, making them more practically useful. 



**BabyAGI** functions similarly - via the integration of GPT-4, a vector store, and LangChain. It creates tasks based on prior outcomes and a set goal. BabyAGI operates in a loop that revolves around four main sub-agents: the Execution Agent, the Task Creation Agent, the Prioritization Agent, and the Context Agent. 

In [3]:
# !pip install faiss-cpu
# !pip install langchain-experimental

In [4]:
# A Code Example of Using BabyAGI

from langchain.embeddings import OpenAIEmbeddings
import faiss
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore

# Define the embedding model
embeddings_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# Initialize the vectorstore
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

In [5]:
from langchain import OpenAI
# from langchain.experimental import BabyAGI
from langchain_experimental.autonomous_agents import BabyAGI

# Setting the goal
goal = "Plan a trip to the Grand Canyon"

# Creating the BabyAGI agent (If max_iterations is None, the agent may go on forever if stuck in loops)
baby_agi = BabyAGI.from_llm(
    llm=OpenAI(model="text-davinci-003", temperature=0),
    vectorstore=vectorstore,
    verbose=False,
    max_iterations=3
)

response = baby_agi({"objective": goal})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list

*****TASK RESULT*****



1. Research the best time to visit the Grand Canyon
2. Book flights to the Grand Canyon
3. Book a hotel near the Grand Canyon
4. Research the best activities to do at the Grand Canyon
5. Make a list of items to pack for the trip
6. Make a budget for the trip
7. Make a list of places to eat near the Grand Canyon
8. Make a list of souvenirs to buy at the Grand Canyon
9. Make a list of places to visit near the Grand Canyon
10. Make a list of emergency contacts to have on hand during the trip

*****TASK LIST*****

2: Research the best way to get to the Grand Canyon from the airport
3: Research the best way to get around the Grand Canyon
4: Research the best places to take pictures at the Grand Canyon
5: Research the best places to go hiking at the Grand Canyon
6: Research the best places to go camping at the Grand Canyon
7: Research the best places to go sightseeing at the Grand Ca

This output reflects the systematic way in which the BabyAGI model approaches tasks: it begins by outlining the tasks, making a to-do list regarding a trip to the Grand Canyon, then it proceeds to complete each task one by one. For each task, it not only lists out the information gained through research but also offers a plan of action or what steps it would take to accomplish the task. The agent also dynamically updates its task list based on new information or steps necessary to accomplish broader tasks, like researching the best ways to get to the Grand Canyon, then breaking it down into more specific sub-tasks.

<hr>
<a class="anchor" id="AutoGPT">
    
## 3. Using AutoGPT with LangChain
    
</a>

<hr>
<a class="anchor" id="simulation">
    
## 4. Agent Simulation Projects: CAMEL and Generative Agents
    
</a>

<hr>
<a class="anchor" id="plan_and_execute">
    
## 5. "Plan and Execute" framework
    
</a>

<hr>
<a class="anchor" id="curr">
    
## 6. Current Insights and Trends of Agents
    
</a>